In [1]:
import os 
import cv2 
import pandas as pd 
from matplotlib import image 
import matplotlib.pyplot as plt 
from sklearn.model_selection import train_test_split

import numpy as np
import re
%matplotlib inline
#from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import keras
from keras.utils import to_categorical
from keras.models import Sequential,Input,Model
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.layers.normalization import BatchNormalization
from keras.layers.advanced_activations import LeakyReLU

import tensorflow as tf

Using TensorFlow backend.


In [2]:
#inicializamos variables
DATA = "G:/datos/expressions/train"
CATEGORIES = ["angry","disgust","fear","happy","neutral","sad","surprise"]
training_data = []
labels=[]
cont_images=[]
index=0
#buscamos en cada carpeta
for category in CATEGORIES:
    cont=0
    path = os.path.join(DATA,category)
    #labels.append(CATEGORIES.index(category))
    #buscamos y cargamos imagenes
    for img in os.listdir(path):
        cont=cont+1
        img_array = plt.imread(os.path.join(path,img))
        #en caso de que haya que cambiar tamaño
        #new_array = cv2.resize(img_array,(48,48))
        training_data.append(img_array)
        labels.append(index)
        #break
        #plt.imshow(new_array)
        #plt.show()
    cont_images.append(cont)
    index=index+1
#print('Etiquetas de cada expresion: ',labels)
print('Numero de imagenes de cada expresion:',cont_images)
print('Numero total de imagenes del set de entrenamiento: ',sum(cont_images))
print('Cantidad de etiquetas creadas (debe ser igual al numero total de imagenes): ',len(labels))

Numero de imagenes de cada expresion: [3993, 436, 4103, 7164, 4982, 4938, 3205]
Numero total de imagenes del set de entrenamiento:  28821
Cantidad de etiquetas creadas (debe ser igual al numero total de imagenes):  28821


In [3]:
#Convertimos las etiquetas y las imagenes en numpy array
y = np.array(labels)
x = np.array(training_data, dtype = np.uint8)

In [4]:
#Mezclar todo y crear los grupos de entrenamiento y testing
train_X,test_X,train_Y,test_Y = train_test_split(x,y,test_size=0.2)
train_X = train_X.reshape(train_X.shape[0], 48, 48, 1)
test_X = test_X.reshape(test_X.shape[0], 48, 48, 1)

print('Training data shape : ', train_X.shape, train_Y.shape)
print('Testing data shape : ', test_X.shape, test_Y.shape)

train_X = train_X.astype('float32')
test_X = test_X.astype('float32')
train_X = train_X / 255.
test_X = test_X / 255.

# Change the labels from categorical to one-hot encoding
train_Y_one_hot = to_categorical(train_Y)
test_Y_one_hot = to_categorical(test_Y)

# Display the change for category label using one-hot encoding
print('Original label:', train_Y[0])
print('After conversion to one-hot:', train_Y_one_hot[0])

train_X,valid_X,train_label,valid_label = train_test_split(train_X, train_Y_one_hot, test_size=0.2, random_state=13)

print(train_X.shape,valid_X.shape,train_label.shape,valid_label.shape)

Training data shape :  (23056, 48, 48, 1) (23056,)
Testing data shape :  (5765, 48, 48, 1) (5765,)
Original label: 3
After conversion to one-hot: [0. 0. 0. 1. 0. 0. 0.]
(18444, 48, 48, 1) (4612, 48, 48, 1) (18444, 7) (4612, 7)


In [5]:
#Cuantas etiquetas unicas hay
classes = np.unique(y)
nClasses = len(classes)
print(classes)

[0 1 2 3 4 5 6]


In [9]:
INIT_LR = 1e-3
epochs = 15
batch_size = 128
 
express_model = Sequential()
express_model.add(Conv2D(32, kernel_size=(3, 3),activation='linear',padding='same',input_shape=(48,48,1)))
express_model.add(LeakyReLU(alpha=0.1))
express_model.add(MaxPooling2D((2, 2),padding='same'))
express_model.add(Dropout(0.5))
 
express_model.add(Flatten())
express_model.add(Dense(32, activation='linear'))
express_model.add(LeakyReLU(alpha=0.1))
express_model.add(Dropout(0.5)) 
express_model.add(Dense(nClasses, activation='softmax'))
 
express_model.summary()
 
express_model.compile(loss=keras.losses.categorical_crossentropy, optimizer=keras.optimizers.Adagrad(lr=INIT_LR, decay=INIT_LR / 100),metrics=['accuracy'])

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 48, 48, 32)        320       
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 48, 48, 32)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 24, 24, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 24, 24, 32)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 18432)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 32)                589856    
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 32)               

In [6]:
from keras.models import Sequential
from keras.layers import Dense , Activation , Dropout ,Flatten
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.metrics import categorical_accuracy
from keras.models import model_from_json

from keras.optimizers import *
from keras.layers.normalization import BatchNormalization

batch_size = 128
epochs = 15
num_class=7

#Shallow CNN model with two Convolution layer & one fully connected layer

# Initialising the CNN
model = Sequential()

# 1 - Convolution
model.add(Conv2D(64,(3,3), padding='same', input_shape=(48, 48,1)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

# 2nd Convolution layer
model.add(Conv2D(128,(5,5), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

# Flattening
model.add(Flatten())

# Fully connected layer 1st layer
model.add(Dense(256))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.25))

model.add(Dense(num_class, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=[categorical_accuracy])

In [7]:
express_trained = model.fit(train_X, train_label, batch_size=batch_size,epochs=epochs,verbose=1,validation_data=(valid_X, valid_label))
# guardamos la red, para reutilizarla en el futuro, sin tener que volver a entrenar
model.save("expressionstrain_mnist.h5py")

Train on 18444 samples, validate on 4612 samples
Epoch 1/15
18444/18444 [==============================] - 180s 10ms/step - loss: 0.4000 - categorical_accuracy: 0.3459 - val_loss: 0.4324 - val_categorical_accuracy: 0.2500
Epoch 2/15
18444/18444 [==============================] - 182s 10ms/step - loss: 0.3201 - categorical_accuracy: 0.4607 - val_loss: 0.4104 - val_categorical_accuracy: 0.2892
Epoch 3/15
 1408/18444 [=>............................] - ETA: 2:39 - loss: 0.2906 - categorical_accuracy: 0.5149

KeyboardInterrupt: 

In [ ]:
test_eval = express_model.evaluate(test_X, test_Y_one_hot, verbose=1)

In [ ]:
print('Test loss:', test_eval[0])
print('Test accuracy:', test_eval[1])